#  Clustering Underserved Areas in Kranj Based on Bus Stop Accessibility

## IMPORTS AND INSTALLS

In [1]:
%%capture
!pip install osmnx folium
!pip install selenium
!pip install tqdm


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
from sklearn.cluster import DBSCAN
import pandas as pd
from google.colab import drive
import os
from shapely.geometry import Point, LineString
from pyproj import Geod
from shapely.ops import nearest_points
from shapely.geometry import Point
import osmnx as ox
from sys import hexversion
import re
import json
from bs4 import BeautifulSoup
from math import radians, sin, cos, sqrt, atan2
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, MultiPoint
from tqdm import tqdm
import selenium

### HELPER FUNCIONS

In [ ]:
def get_center(entry):
  """
  Get the center point of a rectangle defined by bottom-left and top-right coordinates. 
  Args:
      entry (list): A list containing two tuples/lists - bottom-left and top-right coordinates
      Returns:
      tuple: The center coordinates (latitude, longitude)
"""
  bottom_left = entry[0]
  top_right = entry[1]
  center = ((bottom_left[0] + top_right[0])/2, (bottom_left[1] + top_right[1])/2)
  return center

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth's surface.
    Args:
        lat1, lon1: Latitude and longitude of the first point in decimal degrees
        lat2, lon2: Latitude and longitude of the second point in decimal degrees
    Returns:
        float: Distance between the two points in kilometers
    """
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

def get_n_apartments_data(base_path):
  """
Extract apartment data and associated nodes from an HTML file.
Args:
    base_path (str): The base path where the HTML file is located
Returns:
    list: A list of dictionaries containing apartment data and associated nodes
"""

  left_coordinates = []
  right_coordinates = []
  first_time = 1
  # Load the HTML content from your file
  file_path = base_path + 'model_result500.html'
  with open(file_path, 'r', encoding='utf-8') as file:
      html_content = file.read()

  # Extract script elements using BeautifulSoup
  soup = BeautifulSoup(html_content, 'html.parser')
  script_elements = soup.find_all('script')

  # Initialize a list to store the results
  data_entries = []

  # Define the regex patterns
  pattern1 = r'L\.rectangle\(\s*\[\[([0-9.-]+), ([0-9.-]+)\], \[([0-9.-]+), ([0-9.-]+)\]\]'
  pattern2 = r'\{"coordinates": \[([0-9.-]+), ([0-9.-]+)\], "n_apartments": (\d+)\}'

  # Create a dictionary to hold rectangles and their associated apartments
  rectangles = {}

  # Assume city_graph is the graph containing the nodes and their locations
  for script in script_elements:
      if script.string:
          matches = re.findall(pattern1, script.string)
          for match in matches:
              bl_latitude = float(match[0])  # Bottom-left latitude
              bl_longitude = float(match[1])  # Bottom-left longitude
              tr_latitude = float(match[2])  # Top-right latitude
              tr_longitude = float(match[3])  # Top-right longitude
              if first_time:
                  left_coordinates = [bl_latitude, bl_longitude]
                  right_coordinates = [tr_latitude, tr_longitude]
                  first_time = 0
              else:
                  if left_coordinates[0] > bl_latitude or left_coordinates[1] > bl_longitude:
                      left_coordinates[0] = bl_latitude
                      left_coordinates[1] = bl_longitude
                  elif right_coordinates[0] < tr_latitude or right_coordinates[1] < tr_longitude:
                      right_coordinates[0] = tr_latitude
                      right_coordinates[1] = tr_longitude
              center = get_center([[bl_latitude, bl_longitude], [tr_latitude, tr_longitude]])
              # Use the coordinates as a key or identifier
              rectangle_key = (bl_latitude, bl_longitude, tr_latitude, tr_longitude)

              # Initialize node storage
              nodes_in_area = []
              bus_stations = []

              # Check for nodes within bounds
              for node_id, node_data in city_graph.nodes(data=True):
                  node_lat = node_data.get('y')
                  node_lon = node_data.get('x')
                  if bl_latitude <= node_lat <= tr_latitude and bl_longitude <= node_lon <= tr_longitude:
                      station = False
                      for bus_stop in bus_stop_data:
                        if bus_stop['projected_node_id'] == node_id:
                          station = True
                          bus_stations.append(node_id)
                      if not station:
                        nodes_in_area.append(node_id)
                        #print(haversine(node_lat,node_lon, center[0],center[1]))
              # Update rectangle data
              rectangles[rectangle_key] = {
                  'bottom_left': (bl_latitude, bl_longitude),
                  'top_right': (tr_latitude, tr_longitude),
                  'n_apartments': None,  # Placeholder for apartments
                  'center': center,
                  'nodes': nodes_in_area,  # Store nodes in area
                  'bus_stations' : bus_stations
              }

  # Extract apartments and match them
  for script in script_elements:
      if script.string:
          matches = re.findall(pattern2, script.string)
          for match in matches:
              latitude = float(match[0])
              longitude = float(match[1])
              n_apartments = int(match[2])
              apartment_coordinates = (latitude, longitude)

              # Check if the apartments are in the rectangles
              for rectangle_key in rectangles.keys():
                  in_there = 0
                  # You can define a logic to match rectangles with apartment coordinates
                  bl_lat, bl_lon, tr_lat, tr_lon = rectangle_key
                  if bl_lat == latitude and bl_lon == longitude:
                      rectangles[rectangle_key]['n_apartments'] = n_apartments
                      in_there = 1
                      break
              if in_there == 0:
                  print("ERROR")

  # Prepare the final data entries with nodes
  for rectangle_key, data in rectangles.items():
      data_entries.append(data)

  print("left_coordinates",left_coordinates)
  print("right_coordinates",right_coordinates)
  return data_entries

### Illustrate functions for calculating center points and distances 
### Note: These function are not used but are kept for reference

In [4]:

def illustrate_node(m, u, name, color):
    # Get the coordinates of the node
    u_coords = (city_graph.nodes[u]['y'], city_graph.nodes[u]['x'])

    # Add a marker to the map with the given name and color
    folium.Marker(
        location=[u_coords[0], u_coords[1]],
        popup=name,  # Use the variable 'name' instead of the string "name"
        icon=folium.Icon(color=color)  # Use the variable 'color' instead of the string "color"
    ).add_to(m)
    return m

def illustrate_edge(m, u, v,edge_data, key, colors):
    """Illustrate an edge on the Folium map given its nodes and color."""
    if 'geometry' in edge_data:
        # Extract edge geometry
        edge_geom = edge_data['geometry']
        edge_coords = list(edge_geom.coords)

        # Add PolyLine to the map
        folium.PolyLine([(point[1], point[0]) for point in edge_coords], color=colors, weight=5, opacity=0.7).add_to(m)
    else:
        print("No geometry data available for this edge.")

def illustrate_point(m, closest_edge_point, color,name):
  folium.Marker(
          location=[closest_edge_point.y, closest_edge_point.x],
          popup=name,
          icon=folium.Icon(color=color)
      ).add_to(m)



def illustrate_blurry_dot(map_obj, node, color, radius=5, opacity=0.5, blur=0.8):
    """
    Add a blurry dot (CircleMarker) to the map for a given node.
    """
    if 'y' in node and 'x' in node:
        folium.CircleMarker(
            location=(node['y'], node['x']),  # (lat, lon)
            radius=radius,                    # Size of the dot
            color=color,                      # Dot color
            fill=True,
            fill_color=color,
            fill_opacity=opacity,             # Transparency of the dot
            opacity=blur                      # Blur effect
        ).add_to(map_obj)

def read_bus_stops_data(base_path, location= "Kranj, Slovenija"):
  drive.mount('/content/drive')
  # Step 2: Load the GeoJSON file for bus lines into a GeoDataFrame
  gdf_bus_lines = gpd.read_file(os.path.join(base_path, 'bus_stations_lanes.geojson'))

  # Step 3: Check and ensure CRS is in WGS84 (EPSG:4326)
  if gdf_bus_lines.crs != "EPSG:4326":
      gdf_bus_lines = gdf_bus_lines.to_crs(epsg=4326)
  # Step 3: Load bus stops using OSMnx
  city_name = "Kranj, Slovenia"
  bus_stops = ox.features_from_place(city_name, tags={'highway': 'bus_stop'})  # Use 'features_from_place'

  # Convert bus stops to GeoDataFrame
  gdf_bus_stops = gpd.GeoDataFrame(bus_stops, geometry='geometry')

  # Ensure bus stops are also in EPSG:4326
  if gdf_bus_stops.crs != "EPSG:4326":
      gdf_bus_stops = gdf_bus_stops.to_crs(epsg=4326)
  return gdf_bus_lines, gdf_bus_stops

def show_coverage(city_center, city_graph, bus_stop_data):
  bus_iso_map = folium.Map(location=city_center, zoom_start=12)
  iso_polygons = []

  for stop in bus_stop_data:
      node_id = stop['projected_node_id']
      if node_id not in city_graph.nodes:
          continue

      # 1️⃣ Compute reachable nodes within 400 m by road distance
      lengths = nx.single_source_dijkstra_path_length(city_graph, node_id, cutoff=700, weight="length")
      reachable_nodes = list(lengths.keys())

      # 2️⃣ Get coordinates
      coords = [(city_graph.nodes[n]['x'], city_graph.nodes[n]['y']) for n in reachable_nodes]
      if len(coords) < 3:
          continue

      # 3️⃣ Create convex hull from reachable points
      points = [Point(xy) for xy in coords]
      polygon = MultiPoint(points).convex_hull
      iso_polygons.append(polygon)

      # 4️⃣ Draw red dot for bus stop
      folium.CircleMarker(
          location=[city_graph.nodes[node_id]['y'], city_graph.nodes[node_id]['x']],
          radius=1,
          color='red',
          fill=True,
          fill_color='red'
      ).add_to(bus_iso_map)

  # 5️⃣ Overlay polygons with transparent blue
  for poly in iso_polygons:
      folium.GeoJson(
          poly,
          style_function=lambda x: {
              "fillColor": "blue",
              "color": None,
              "fillOpacity": 0.07,  # adjust transparency to visualize saturation
          },
      ).add_to(bus_iso_map)

  return bus_iso_map



# Class Network_Object:


*   Makes graph from OpenStreetMap based on city
*   Projects bus stops to the graph so that they are a part of the graph



In [ ]:
class Network_Object():
  """
  A class to manage and manipulate transportation networks and bus stop data.
"""
  def __init__(self,G_drive=None, city_graph=None, bus_stop_data=None):
        self.G_drive = G_drive
        self.city_graph = city_graph
        self.bus_stop_data = bus_stop_data
  def make_graph(self, place_name, types):
      """
      Download and combine street networks of specified types into a single graph.

      Parameters:
      place_name (str): The name of the location (e.g., "Kranj, Slovenia").
      types (list): A list of network types to download (e.g., ['drive', 'walk', 'bike']).

      Returns:
      networkx.Graph: A combined undirected graph containing all specified networks.
      """
      graphs = []

      for network_type in types:
          G = ox.graph_from_place(place_name, network_type=network_type)
          graphs.append(G)

      # Combine all graphs
      G_combined = nx.compose_all(graphs)
      # Convert to undirected
      return G_combined.to_undirected()


  def project_point_to_edge(self,point, edge_geometry):
    """
    Project a point onto a given edge geometry.
    
    Args:
        point (shapely.geometry.Point): The point to be projected.
        edge_geometry (shapely.geometry.LineString): The geometry of the edge.

    Returns:
        shapely.geometry.Point: The projected point on the edge.
    """
    # Project the point onto the edge geometry
    projected_point = edge_geometry.interpolate(edge_geometry.project(point))
    # Return the projected point on the edge
    return projected_point

  def project_point_to_closest_non_footpath_edge(self,graph,G_graph, point):
    """
    Project a point onto the closest non-footpath edge in the graph.
    Args:
        graph (networkx.Graph): The graph to search for edges.
        point (shapely.geometry.Point): The point to be projected.
    Returns:
        tuple: A tuple containing the closest edge (as a dictionary) and the projected point (shapely.geometry.Point).
    """
    # Find the nearest edge to the point in the graph
    uO, vO, key = ox.distance.nearest_edges(graph, point.x, point.y, return_dist=False)
    min_u = None
    min_v = None
    closest_edge = None
    min_distance = float('inf')
    projected_point_min = None
    list_bad = ["footway","pedestrian","cycleway","living_street","bridleway","motorway","motorway_link","service"]
    edge_data = graph.get_edge_data(uO, vO)

    if edge_data[0]['highway'] not in list_bad and 'geometry' in edge_data[0]:
        projected_point = self.project_point_to_edge(point, edge_data[0]['geometry'])
        projected_point_min = self.project_point_to_edge(point, edge_data[0]['geometry'])
        min_distance = projected_point.distance(point)
        closest_edge = edge_data[0]
        min_u = uO
        min_v = vO
    # Get all neighbors of uO and vO
    neighbors = list(graph.neighbors(uO)) + list(graph.neighbors(vO))


    # Iterate through each neighbor and their neighboring edges
    for neighbor in neighbors:
        list_v_neighbors = list(graph.neighbors(neighbor))
        for new_neighbor in list_v_neighbors:
            edge_data = graph.get_edge_data(neighbor, new_neighbor)

            if edge_data:  # Check if edge data exists
                edge = edge_data[0]

                # Exclude footway edges
                if edge['highway'] not in list_bad and 'geometry' in edge:
                    #print("edge",edge)
                   # print("edger", edge)
                    projected_point = self.project_point_to_edge(point, edge['geometry'])
                    distance = projected_point.distance(point)

                    # Update closest edge if found a shorter one
                    if distance < min_distance:
                        min_distance = distance
                        closest_edge = edge
                        projected_point_min = projected_point
                        min_u = neighbor
                        min_v = new_neighbor
    #print(projected_point_min)
    uO, vO, key = ox.distance.nearest_edges(G_graph, point.x, point.y, return_dist=False)
    edge_data = G_graph.get_edge_data(uO, vO)
    if edge_data[0]['highway'] not in list_bad and 'geometry' in edge_data[0]:
        projected_point = self.project_point_to_edge(point, edge_data[0]['geometry'])
        distance = projected_point.distance(point)
        if distance < min_distance:
            min_distance = distance
            closest_edge = edge_data[0]
            projected_point_min = projected_point
            min_u = uO
            min_v = vO
    #print(projected_point_min)
    return [min_u,min_v,closest_edge] , projected_point_min

    # Create an empty list to store the information about each bus stop
  def is_node_on_edge(self,node, edge_geometry):
    """
    Check if a node is located on a given edge geometry.
    Args:
        node (dict): A dictionary containing node attributes, including 'x' and 'y' coordinates.
        edge_geometry (shapely.geometry.LineString): The geometry of the edge.
    """

    # Convert the node to a point geometry
    node_point = Point(node['x'], node['y'])
    # Check if the point intersects with the edge (LineString)
    return edge_geometry.intersects(node_point)


  def find_nodes_on_edge(self,city_graph, closest_edge_geometry):
    """
    Find all nodes in the city_graph that are located on the given edge geometry.
    Args:
        city_graph (networkx.Graph): The graph containing nodes and edges.
        closest_edge_geometry (shapely.geometry.LineString): The geometry of the edge.
    Returns:
        list: A list of node IDs that are located on the edge.
    """
    nodes_on_edge = []

    # Iterate over all nodes in city_graph
    for node, data in city_graph.nodes(data=True):
        # Check if the node is part of the edge by checking its position
        for u, v, key, edge_data in city_graph.edges(node, data=True, keys=True):
            if edge_data.get('geometry') is not None:
                edge_geometry = LineString(edge_data['geometry'].coords)
                if self.is_node_on_edge(data, closest_edge_geometry):
                    nodes_on_edge.append(node)  # Add the node to the list if it is on the edge

    return nodes_on_edge
  def find_best_nodes_around_projected_point(self,city_graph, u, v, projected_point):
    """
    Find the best pair of nodes (u, v) around a projected point on an edge.
    Args:
        city_graph (networkx.Graph): The graph containing nodes and edges.
        u (int): The starting node of the edge.
        v (int): The ending node of the edge.
        projected_point (shapely.geometry.Point): The projected point on the edge.
    Returns:
        tuple: A tuple containing the best pair of nodes (best_u, best_v).
    """
    # Get the edge data between u and v
    edge_data = city_graph.get_edge_data(u, v)

    edge_geometry = edge_data[0]['geometry']
    nodes_on_same_street = self.find_nodes_on_edge(city_graph, edge_geometry)
    best_u = u
    best_v = v
    min_distance = float('inf')
    print("nodes_on_same", nodes_on_same_street)
    # Iterate over all nodes on the same street
    for node in nodes_on_same_street:
        # Get the neighbors of the node
        neighbors = list(city_graph.neighbors(node))

        # Iterate over the neighbors
        for neighbor in neighbors:
            # Check if the neighbor is also on the same street
            if neighbor in nodes_on_same_street:
                # Calculate the distance from the projected point to the line segment between node and neighbor
                line_segment = LineString([Point(city_graph.nodes[node]['x'], city_graph.nodes[node]['y']),
                                            Point(city_graph.nodes[neighbor]['x'], city_graph.nodes[neighbor]['y'])])
                distance = line_segment.distance(projected_point)

                # Update the best nodes if this pair is closer to the projected point
                if distance < min_distance:
                    min_distance = distance
                    best_u = node
                    best_v = neighbor

    return best_u, best_v

  def get_distance_path(self, city_graph, u, v, closest_edge_point, edge_data):
    """ 
    Calculate distances and paths from a projected point on an edge to its nodes u and v.
    Args:
        city_graph (networkx.Graph): The graph containing nodes and edges.
        u (int): The starting node of the edge.
        v (int): The ending node of the edge.
        closest_edge_point (shapely.geometry.Point): The projected point on the edge.
        edge_data (dict): The data associated with the edge between u and v.
        Returns:
        dict: A dictionary containing distances to u and v, and LineStrings to u and v.
    """
    # Initialize the geod object for calculating distances
    geod = Geod(ellps='WGS84')

    # Coordinates of nodes u and v
    u_coords = (city_graph.nodes[u]['x'], city_graph.nodes[u]['y'])
    v_coords = (city_graph.nodes[v]['x'], city_graph.nodes[v]['y'])
    u_point = Point(u_coords)
    v_point = Point(v_coords)

    # Convert closest_edge_point to Point for comparison
    closest_edge_point = Point(closest_edge_point.x, closest_edge_point.y)

    # Initialize distances
    distance_to_u = 0
    distance_to_v = 0

    # Check if 'geometry' exists in edge_data
    if 'geometry' in edge_data:
        edge_geometry = edge_data['geometry']
    else:
        # If no geometry exists, create an approximation between nodes u and v
        edge_geometry = LineString([u_point, v_point])

    coordinates_to_u = []  # Store points leading to bus stop
    coordinates_to_v = []  # Store points leading from bus stop to node V
    to_u = True  # Flag to track whether to append to u or v

    for i in range(len(edge_geometry.coords) - 1):
        coord1 = Point(edge_geometry.coords[i])
        coord2 = Point(edge_geometry.coords[i + 1])

        # Create a LineString for the segment
        segment = LineString([coord1, coord2])

        # Check if the closest_edge_point is within the distance of the segment
        if segment.distance(closest_edge_point) < 1e-6:  # Use a small tolerance
            # Calculate distances to u and v from the closest edge point
            distance_to_u += geod.inv(closest_edge_point.x, closest_edge_point.y, coord1.x, coord1.y)[2]  # Distance in meters
            distance_to_v += geod.inv(closest_edge_point.x, closest_edge_point.y, coord2.x, coord2.y)[2]  # Distance in meters
            to_u = False  # Switch to appending to coordinates_to_v
            coordinates_to_u.append(coord1)  # Add the last coordinate to the path to V
            coordinates_to_u.append(closest_edge_point)
            coordinates_to_v.append(closest_edge_point)
            coordinates_to_v.append(coord2)
        else:
            if to_u:
                coordinates_to_u.append(coord1)  # Append coord1 for path to U
                distance_to_u += geod.inv(coord1.x, coord1.y, coord2.x, coord2.y)[2]  # Add the distance to U
            else:
                distance_to_v += geod.inv(coord1.x, coord1.y, coord2.x, coord2.y)[2]  # Add distance to V
                coordinates_to_v.append(coord2)  # Append coord2 for path to V

    # Create LineStrings from the lists of coordinates
    line_to_u = LineString(coordinates_to_u)  # LineString from U to bus stop
    line_to_v = LineString(coordinates_to_v)  # LineString from bus stop to V

    return {
        "distance_to_u": distance_to_v,
        "distance_to_v": distance_to_u,
        "line_to_u": line_to_u,
        "line_to_v": line_to_v
    }

  def add_new_node(self,city_graph,bus_stop_point,name):
    """
    Add a new node to the city_graph at the specified bus_stop_point.
    Args:
        city_graph (networkx.Graph): The graph to which the node will be added.
        bus_stop_point (shapely.geometry.Point): The location of the new bus stop.
        name (str): The name of the bus stop.
    Returns:
        int: The ID of the newly added node.
    """
    node_data = {
        "y":bus_stop_point.y,
        "x":bus_stop_point.x,
        "highway":"bus_stop",
        "name": name,
        "street_count": 2
    }
    node_id = max(city_graph.nodes, default=-1) + 1 # Get a new node ID
    city_graph.add_node(node_id, **node_data)  # Add the node with attributes
    #print("Node attributes:", city_graph.nodes[node_id])
    #print(f"Added new node: {node_data} with ID: {node_id}")
    return node_id
    #node_data {'y': 46.2376896, 'x': 14.3498662, 'highway': 'traffic_signals', 'street_count': 5}

  def add_new_edge(self,city_graph, node_from, node_to, data, edge_data_prv):
    """
    Add a new edge to the city_graph between node_from and node_to with specified data.
    Args:
        city_graph (networkx.Graph): The graph to which the edge will be added.
        node_from (int): The starting node of the edge.
        node_to (int): The ending node of the edge.
        data (dict): A dictionary containing edge attributes such as 'length' and 'route'.
        edge_data_prv (dict): A dictionary containing existing edge attributes to copy from.
    """

    edge_data = edge_data_prv.copy()  # Copy existing edge data to avoid modifying the original
    edge_data['length'] = data['length']
    edge_data['geometry'] = data['route']  # Ensure this is a LineString or appropriate geometry type

    # Add the edge to the graph
    city_graph.add_edge(node_from, node_to, **edge_data)

    #print(f"Added edge from {node_from} to {node_to} with data: {edge_data}")

  def Project_Bus_to_graph(self,gdf_bus_stops,city_graph,G_drive):
    """
    Project bus stops onto the city graph and update the graph with new nodes and edges.
    """
    bus_stop_data = []
    copy_city_graph = city_graph.copy()
    for stop_idx, stop_row in gdf_bus_stops.iterrows():
        # Step 1: Project the stop onto G_drive (road network)
        #projected_point, (u, v, edge_data) = project_stop_to_nearest_edge(city_graph, stop_row['geometry'])

        closest_edge, projected_point = self.project_point_to_closest_non_footpath_edge(copy_city_graph, G_drive, stop_row['geometry'])
        if projected_point is not None:
            #print("projected_point",projected_point)
            #print("closest_edge",closest_edge)
            u =  closest_edge[0]
            v =  closest_edge[1]
            edge_data = closest_edge[2]
            #print("closest_edge",closest_edge[0])
            u =  closest_edge[0]
            v =  closest_edge[1]
            edge_data = closest_edge[2]
            best_u, best_v = self.find_best_nodes_around_projected_point(copy_city_graph,u,v,projected_point)
            print(best_u, best_v)
            print("best",copy_city_graph.get_edge_data(best_u,best_v))
            print("not",copy_city_graph.get_edge_data(u,v))
            print("projected point", projected_point)
            #print(len(copy_city_graph.get_edge_data(best_u,best_v)[0]['geometry'].coords))
            if 'geometry' not in city_graph.get_edge_data(best_u,best_v)[0]:
              print("no geometry")
              distance_data = self.get_distance_path(city_graph, u, v, projected_point, city_graph.get_edge_data(u,v)[0])
            # Step 2: Compute distances and paths using city_graph
            else:
              distance_data = self.get_distance_path(city_graph, best_u, best_v, projected_point, city_graph.get_edge_data(best_u,best_v)[0])

            # Step 3: Add the projected point as a new node in city_graph
            projected_node_id = self.add_new_node(city_graph, projected_point, name=f"Projected Stop {stop_idx}")

            # Step 4: Check if the edge exists before removing it
            #if city_graph.has_edge(u, v):
                # Remove the old edge in city_graph (optional)
                #city_graph.remove_edge(u, v)

            # Step 5: Add new edges in city_graph
            self.add_new_edge(city_graph, u, projected_node_id,
                        {"length": distance_data["distance_to_u"], "route": distance_data["line_to_u"]}, edge_data)
            self.add_new_edge(city_graph, projected_node_id, v,
                        {"length": distance_data["distance_to_v"], "route": distance_data["line_to_v"]}, edge_data)

            print(f"Added projected bus stop {projected_node_id} at {projected_point} between {u} and {v}")

            # Step 6: Store the bus stop's data
            bus_stop_data.append({
                'stop_idx': stop_idx,  # Bus stop index
                'projected_point': projected_point,  # Projected point
                'projected_node_id': projected_node_id  # New node ID
            })

        else:
            print(f"Error: Could not project bus stop at index {stop_idx}")
            break
        print("edge new data",city_graph.get_edge_data(u,projected_node_id))

    self.city_graph = city_graph
    self.G_drive = G_drive
    self.bus_stop_data = bus_stop_data
    return city_graph, bus_stop_data


## Make graphs

In [6]:
place_name = "Kranj, Slovenia"
types = ['drive', 'walk', 'bike']
network_Obj = Network_Object()
G_combined = network_Obj.make_graph(place_name, types)

In [7]:

city_graph = G_combined
city_center = ox.geocode(place_name)
# Step 2: Get the number of nodes and edges
num_nodes = len(city_graph.nodes)  # Number of nodes
num_edges = len(city_graph.edges)  # Number of edges

print(f"Number of nodes (intersections, stops): {num_nodes}")
print(f"Number of edges (streets): {num_edges}")


Number of nodes (intersections, stops): 8540
Number of edges (streets): 12314


In [8]:
gdf_roads = ox.graph_to_gdfs(city_graph, nodes=False, fill_edge_geometry=True)
place_name = "Kranj, Slovenia"
type = ['drive']
G_drive = network_Obj.make_graph(place_name, type)
G_drive = ox.project_graph(G_drive, to_crs=gdf_roads.crs)
city_graph = G_combined

In [9]:
gdf_bus_lines, gdf_bus_stops = read_bus_stops_data(base_path = "/content/drive/My Drive/fri/E7/Public_nodes/", location= "Kranj, Slovenija") # base_path, location= "Kranj, Slovenija"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
city_graph, bus_stop_data = network_Obj.Project_Bus_to_graph(gdf_bus_stops,city_graph,G_drive)

nodes_on_same [1735800413, 1735800413, 1735800413, 4428912535, 4428912535, 4428912535, 153563389, 153563389, 153563389, 153563389, 4428912536, 9683541339, 9683541339, 9683541339, 9683541339, 9683541339, 12101165029, 12101165029]
153563389 12101165029
best {0: {'osmid': [15461547, 163605423], 'highway': 'residential', 'name': 'Kolodvorska cesta', 'oneway': False, 'reversed': False, 'length': np.float64(58.539271175555925), 'geometry': <LINESTRING (14.35 46.238, 14.35 46.238, 14.349 46.238, 14.349 46.238, 14.34...>}}
not {0: {'osmid': [165222953, 15461547, 195155734, 163605423], 'highway': 'residential', 'lanes': '3', 'name': 'Kolodvorska cesta', 'oneway': False, 'reversed': False, 'length': np.float64(178.99649963015202), 'geometry': <LINESTRING (14.35 46.238, 14.35 46.238, 14.35 46.238, 14.35 46.238, 14.349 ...>}}
projected point POINT (14.349441293888448 46.23841115588016)
Added projected bus stop 13319045952 at POINT (14.349441293888448 46.23841115588016) between 1735800413 and 44289

In [12]:
map_coverage  = show_coverage(city_center, city_graph, bus_stop_data)
map_coverage

In [13]:
data_entries = get_n_apartments_data(base_path = '/content/drive/My Drive/fri/E7/Public_nodes/')

left_coordinates [46.206417, 14.301414]
right_coordinates [46.289988, 14.411106]


# Cluster_obj class


*   Given the graph, bus stops hose_nodes, distance produces the clusters of unreachable nodes
*   unreachable nodes have center node and all nodes within the cluster that are not reachable from a given distance



In [ ]:
class Cluster_Obj():
  """
  A class to manage clustering of apartment areas and analyze reachability.
  """
  def __init__(self,G_drive=None, city_graph=None, bus_stop_data=None,house_nodes = None, distance = None,data_entries = None):
    self.G_drive = G_drive
    self.city_graph = city_graph
    self.bus_stop_data = bus_stop_data
    self.data_entries = data_entries
    self.houses_nodes  = None
    self.clusters = None
    self.unreachables = None
    self.distance = distance
    self.clean_nodes = self.get_clean_nodes()
    self.existing_reachable_nodes = self.get_reachables()
    self.house_nodes = house_nodes

  def get_reachables(self):
    """
    Calculate and return the set of nodes reachable from existing bus stops within the specified distance.
    """
    print("calculating existing reachable nodes")
    existing_reachable_nodes = set()
    for stop in bus_stop_data:
        node_id = stop['projected_node_id']
        if node_id not in city_graph.nodes:
            continue
        lengths = nx.single_source_dijkstra_path_length(city_graph, node_id, cutoff=self.distance, weight="length")
        existing_reachable_nodes.update(lengths.keys())
    return existing_reachable_nodes
  def get_n_apartmetns(self, point):
    """
    Get the number of apartments and center for a given point based on data entries.
    """
    dicto = {}
    for entry in self.data_entries:
      # Get bottom-left and top-right coordinates and number of apartments
      bottom_left = entry['bottom_left']
      top_right = entry['top_right']
      if(point[0] >= bottom_left[0] and point[1] >= bottom_left[1] and point[0] <=top_right[0] and point[1] <= top_right[1]):
        n_apartments = entry['n_apartments']
        center = entry['center']
        #print("n_apartments",n_apartments,"center",center)
        return([n_apartments,center])
    return ([0,-1])
  def get_clean_nodes(self):
    """
    Get nodes from the city graph that have associated apartment information.
    """
    print("getting clean nodes")
    clean_nodes_dict = {}  # Use a different name to avoid overwriting
    for node, data in self.city_graph.nodes(data=True):
        node_lat = data['y']  # Latitude of the city_graph node
        node_lon = data['x']  # Longitude of the city_graph node
        info_area = self.get_n_apartmetns([node_lat, node_lon])

        if info_area[0] > 0:  # If apartments info is greater than 0
            clean_nodes_dict[node] = data  # Add the node to the clean dictionary
            #print(node)  # Debugging print to check the node being added

    return clean_nodes_dict  # Return the cleaned nodes dictionary
  def get_center(self,entry):
    """
    Calculate the center point of a given entry based on its bottom-left and top-right coordinates.
    """
    bottom_left = entry[0]
    top_right = entry[1]
    center = ((bottom_left[0] + top_right[0])/2, (bottom_left[1] + top_right[1])/2)
    return center
  def haversine(self,lat1, lon1, lat2, lon2):
      """
      Calculate the great-circle distance between two points on the Earth's surface using the Haversine formula.
      Args:
          lat1, lon1: Latitude and longitude of the first point in decimal degrees.
          lat2, lon2: Latitude and longitude of the second point in decimal degrees.
      Returns:
          Distance between the two points in kilometers.
      """
      # Radius of the Earth in kilometers
      R = 6371.0

      # Convert latitude and longitude from degrees to radians
      lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

      # Difference in coordinates
      dlat = lat2 - lat1
      dlon = lon2 - lon1

      # Haversine formula
      a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
      c = 2 * atan2(sqrt(a), sqrt(1 - a))

      # Distance in kilometers
      distance = R * c
      return distance
  def find_neighbor_areas(self,city_graph,node,limit):
    """
    Find neighboring areas within a specified distance limit from a given node.
    """
    node_entry = self.data_entries[0]
    found = 0
    entries = []
    for entry in self.data_entries:
      if node in entry['nodes']: #or node in entry['must_points']:
        node_entry = entry
        found = 1
        break
    if found == 0:
      print("ERROR")
      return {}
    for entry in self.data_entries:
      center = entry['center']
      #print(haversine(center[0],center[1], node_entry['center'][0],node_entry['center'][1]))
      if(self.haversine(center[0],center[1], node_entry['center'][0],node_entry['center'][1]) <= limit + 0.125):
        entries.append(entry)
    return entries

  def make_unreachables(self, distance_, distance_areas): #700,0.85
    """
    Identify unreachable nodes in the city graph based on distance criteria.
    """
    print("calculating unreachables")
    unreachables_ = {}
    length_clean_nodes = len(self.clean_nodes)
    for idx, node in enumerate(self.clean_nodes.items()):
      if idx % 500 == 0:
        tqdm.write(f"Processed {idx}/{length_clean_nodes}")
      if self.city_graph.nodes[node[0]].get("name") != None:
        clean_name = self.city_graph.nodes[node[0]].get("name").split(" (")[0]
        if clean_name == "Projected Stop":
          print("k")
          break
      unreachable = True
      neighbors = self.find_neighbor_areas(self.city_graph,node[0],distance_areas)
      #print(neighbors)
      for neighbor in neighbors:
        for station in neighbor['bus_stations']:
          if station != node[0]:
            distance = nx.shortest_path_length(self.city_graph, source=node[0], target=station, weight='length', method='dijkstra')
            if distance < distance_:
              unreachable = False
              #print(distance)
              break
          else:
            print("wopi station")
            unreachable = False
            break
        if unreachable == False:
            break
      if unreachable == True:
        unreachables_[node[0]] = node[1]
    self.unreachables = unreachables_
    return unreachables_

  def get_houses_nodes(self,location = "Kranj, Slovenia"):
    """
    Retrieve house nodes from the specified location using OSMnx.
    """
    
    gdf_buildings = ox.features_from_place( location, tags={"building": True})
    houses = gdf_buildings[gdf_buildings["building"].isin(["house", "residential", "apartments"])]

    # Convert houses to nearest graph node for fast lookup
    house_nodes = set()
    for _, row in houses.iterrows():
        house_point = row.geometry.centroid  # use centroid in case of polygon
        nearest_node = ox.distance.nearest_nodes(self.city_graph, X=house_point.x, Y=house_point.y)
        house_nodes.add(nearest_node)
    self.house_nodes = house_nodes
    return house_nodes

  def cluster_unreachable_nodes_road_distance_H(self,radius=400):
      """
      Cluster unreachable nodes based on road distance and analyze coverage of new houses.
      """
      if self.unreachables is None:
        unreachables = self.make_unreachables(self.distance, distance_areas=0.85)
      all_unreachable_nodes = set(self.unreachables)
      if self.houses_nodes is None:
        self.houses_nodes = self.get_houses_nodes()

      # Step 1: Collect only areas that contain unreachable nodes
      relevant_areas = [entry for entry in self.data_entries if any(n in all_unreachable_nodes for n in entry['nodes'])]

      # Step 2: Collect all unreachable nodes from these areas
      node_positions = {}
      for area in relevant_areas:
          for n in area['nodes']:
              if n in all_unreachable_nodes and n in self.city_graph.nodes:
                  node_positions[n] = (self.city_graph.nodes[n]['x'], self.city_graph.nodes[n]['y'])

      if not node_positions:
          print("No unreachable nodes found for clustering.")
          return {}

      # Step 3: Compute pairwise road distances
      print("all set for clustering")
      node_ids = list(node_positions.keys())
      distance_matrix = np.full((len(node_ids), len(node_ids)), np.inf)
      for i, node_a in enumerate(node_ids):
          distances = nx.single_source_dijkstra_path_length(self.city_graph, source=node_a, cutoff=radius, weight="length")
          for j, node_b in enumerate(node_ids):
              if node_b in distances:
                  distance_matrix[i, j] = distances[node_b]

      # Replace np.inf with a large number
      distance_matrix[distance_matrix == np.inf] = 10 * radius

      # Step 4: First DBSCAN pass (rough grouping)
      clustering = DBSCAN(eps=radius, min_samples=1, metric="precomputed").fit(distance_matrix)

      # Step 5: Group unreachable nodes by cluster label
      clusters = {}
      for idx, label in enumerate(clustering.labels_):
          clusters.setdefault(label, {"nodes": [], "center_node": None})
          clusters[label]["nodes"].append(node_ids[idx])

      # Step 6: Refine clusters by ensuring max distance <= radius AND new coverage
      refined_clusters = {}
      cluster_id = 0

      for label, cluster_data in clusters.items():
          nodes = cluster_data["nodes"]
          points = [Point(node_positions[n]) for n in nodes]
          centroid = MultiPoint(points).centroid

          # Find the node closest to centroid
          center_node = min(
              nodes, key=lambda n: Point(node_positions[n]).distance(centroid)
          )

          # Compute real road distances from center node
          distances = nx.single_source_dijkstra_path_length(
              self.city_graph, source=center_node, weight="length", cutoff=radius * 2
          )
          valid_nodes = [n for n in nodes if distances.get(n, np.inf) <= radius]

          # Step 6a: Check new houses covered by this center node
          reachable_nodes_set = set(distances.keys())
          exclusive_nodes = reachable_nodes_set - self.existing_reachable_nodes
          new_houses = [h for h in self.houses_nodes if h in exclusive_nodes]

          # Only keep cluster if it covers at least one new house
          if valid_nodes and len(new_houses) > 0:
              refined_clusters[cluster_id] = {
                  "nodes": valid_nodes,
                  "center_node": center_node,
                  "new_houses_count": len(new_houses)
              }
              cluster_id += 1
      self.clusters = refined_clusters
      return refined_clusters


In [23]:
existing_reachable_nodes = set()
for stop in bus_stop_data:
    node_id = stop['projected_node_id']
    if node_id not in city_graph.nodes:
        continue
    lengths = nx.single_source_dijkstra_path_length(city_graph, node_id, cutoff=400, weight="length")
    existing_reachable_nodes.update(lengths.keys())

In [ ]:
Cluster_400_obj = Cluster_Obj(G_drive, city_graph, bus_stop_data, distance = 400,data_entries=data_entries)


In [25]:
unreachable_clusters_400_H = Cluster_400_obj.cluster_unreachable_nodes_road_distance_H(radius=400)

calculating unreachables
Processed 0/4678
Processed 500/4678
Processed 1000/4678
Processed 1500/4678
Processed 2000/4678
Processed 2500/4678
Processed 3000/4678
Processed 3500/4678
Processed 4000/4678
k
all set for clustering


In [26]:
Cluster_700_obj  = Cluster_Obj(G_drive, city_graph, bus_stop_data, distance = 700,data_entries=data_entries,house_nodes=Cluster_400_obj.house_nodes)
unreachable_clusters_700_H = Cluster_700_obj.cluster_unreachable_nodes_road_distance_H(radius=700)

getting clean nodes
calculating existing reachable nodes
calculating unreachables
Processed 0/4678
Processed 500/4678
Processed 1000/4678
Processed 1500/4678
Processed 2000/4678
Processed 2500/4678
Processed 3000/4678
Processed 3500/4678
Processed 4000/4678
k
all set for clustering


In [31]:
Cluster_300_obj  = Cluster_Obj(G_drive, city_graph, bus_stop_data, distance = 300,data_entries=data_entries,house_nodes=Cluster_400_obj.house_nodes)
unreachable_clusters_300_H = Cluster_300_obj.cluster_unreachable_nodes_road_distance_H(radius=300)
Cluster_500_obj = Cluster_Obj(G_drive, city_graph, bus_stop_data, distance = 500,data_entries=data_entries,house_nodes=Cluster_400_obj.house_nodes)
unreachable_clusters_500_H = Cluster_500_obj.cluster_unreachable_nodes_road_distance_H(radius=500)

getting clean nodes
calculating existing reachable nodes
calculating unreachables
Processed 0/4678
Processed 500/4678
Processed 1000/4678
Processed 1500/4678
Processed 2000/4678
Processed 2500/4678
Processed 3000/4678
Processed 3500/4678
Processed 4000/4678
k
all set for clustering
getting clean nodes
calculating existing reachable nodes
calculating unreachables
Processed 0/4678
Processed 500/4678
Processed 1000/4678
Processed 1500/4678
Processed 2000/4678
Processed 2500/4678
Processed 3000/4678
Processed 3500/4678
Processed 4000/4678
k
all set for clustering


In [16]:
def get_houses_nodes(city_graph,location = "Kranj, Slovenia"):
    gdf_buildings = ox.features_from_place( location, tags={"building": True})
    houses = gdf_buildings[gdf_buildings["building"].isin(["house", "residential", "apartments"])]

    # Convert houses to nearest graph node for fast lookup
    house_nodes = set()
    for _, row in houses.iterrows():
        house_point = row.geometry.centroid  # use centroid in case of polygon
        nearest_node = ox.distance.nearest_nodes(city_graph, X=house_point.x, Y=house_point.y)
        house_nodes.add(nearest_node)

    return house_nodes
house_nodes = get_houses_nodes(city_graph)

In [27]:
import networkx as nx
from shapely.geometry import Point, MultiPoint
import folium

# Create base map
bus_iso_map = folium.Map(location=city_center, zoom_start=13)

# --- 1️⃣ Original bus stops in blue ---
for stop in bus_stop_data:
    node_id = stop['projected_node_id']
    if node_id not in city_graph.nodes:
        continue

    # Compute reachable nodes within 400 m
    lengths = nx.single_source_dijkstra_path_length(city_graph, node_id, cutoff=400, weight="length")
    reachable_nodes = list(lengths.keys())
    coords = [(city_graph.nodes[n]['x'], city_graph.nodes[n]['y']) for n in reachable_nodes]
    if len(coords) < 3:
        continue

    # Convex hull
    points = [Point(xy) for xy in coords]
    polygon = MultiPoint(points).convex_hull

    # Draw bus stop (red dot)
    folium.CircleMarker(
        location=[city_graph.nodes[node_id]['y'], city_graph.nodes[node_id]['x']],
        radius=2,
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(bus_iso_map)

    # Draw reachable area in blue
    folium.GeoJson(
        polygon,
        style_function=lambda x: {
            "fillColor": "blue",
            "color": None,
            "fillOpacity": 0.07,
        },
    ).add_to(bus_iso_map)

# --- 2️⃣ New "stations" from filtered_unreachable_clusters in red ---
for label, cluster_data in unreachable_clusters_400_H.items():
    center_node = cluster_data["center_node"]
    if center_node not in city_graph.nodes:
        continue

    # Compute reachable nodes within 400 m
    lengths = nx.single_source_dijkstra_path_length(city_graph, center_node, cutoff=400, weight="length")
    reachable_nodes = list(lengths.keys())
    coords = [(city_graph.nodes[n]['x'], city_graph.nodes[n]['y']) for n in reachable_nodes]
    if len(coords) < 3:
        continue

    # Convex hull
    points = [Point(xy) for xy in coords]
    polygon = MultiPoint(points).convex_hull

    # Draw new station (red dot)
    folium.CircleMarker(
        location=[city_graph.nodes[center_node]['y'], city_graph.nodes[center_node]['x']],
        radius=3,
        color='green',
        fill=True,
        fill_color='green'
    ).add_to(bus_iso_map)

    # Draw reachable area in red
    folium.GeoJson(
        polygon,
        style_function=lambda x: {
            "fillColor": "green",
            "color": None,
            "fillOpacity": 0.4,
        },
    ).add_to(bus_iso_map)

bus_iso_map


In [18]:
import networkx as nx
from shapely.geometry import Point, MultiPoint
import folium

def plot_top_clusters_fixed(
    city_center,
    city_graph,
    house_nodes,
    unreachable_clusters,
    new_cutoff=700,
    min_houses=1,
    top_k=5,
    colors_list=None
):
    """
    Plots a Folium map showing only the top-k clusters based on new houses covered.
    - Black: cluster center
    - Colored nodes: cluster nodes
    - Red polygons: convex hull of cluster nodes (not full reach)
    """

    if colors_list is None:
        colors_list = ['yellow', 'purple', 'white', 'orange', 'green']

    # --- Step 1: Compute houses reachable by each cluster ---
    cluster_house_counts = {}
    cluster_polygons = {}

    for label, cluster_data in unreachable_clusters.items():
        center_node = cluster_data["center_node"]
        nodes_in_cluster = [n for n in cluster_data["nodes"] if n in city_graph.nodes]
        if not nodes_in_cluster:
            continue

        # Count houses only in cluster nodes
        houses_covered = [h for h in house_nodes if h in nodes_in_cluster]
        if len(houses_covered) < min_houses:
            continue

        cluster_house_counts[label] = len(houses_covered)

        # Build polygon only from cluster nodes
        coords = [(city_graph.nodes[n]['x'], city_graph.nodes[n]['y']) for n in nodes_in_cluster]
        points = [Point(xy) for xy in coords]
        cluster_polygons[label] = MultiPoint(points).convex_hull

    # --- Step 2: Select top-k clusters ---
    top_clusters = sorted(cluster_house_counts.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # --- Step 3: Create Folium map ---
    folium_map = folium.Map(location=city_center, zoom_start=12)

    for idx, (label, count) in enumerate(top_clusters):
        cluster_data = unreachable_clusters[label]
        nodes_in_cluster = [n for n in cluster_data["nodes"] if n in city_graph.nodes]
        center_node = cluster_data["center_node"]

        # Draw cluster nodes
        for node in nodes_in_cluster:
            folium.CircleMarker(
                location=[city_graph.nodes[node]['y'], city_graph.nodes[node]['x']],
                radius=1,
                color=colors_list[idx % len(colors_list)],
                fill=True,
                fill_color=colors_list[idx % len(colors_list)],
            ).add_to(folium_map)

        # Draw cluster center
        folium.CircleMarker(
            location=[city_graph.nodes[center_node]['y'], city_graph.nodes[center_node]['x']],
            radius=3,
            color='black',
            fill=True,
            fill_color='black',
            popup=f"Cluster {label}, houses: {count}"
        ).add_to(folium_map)

        # Draw cluster polygon
        folium.GeoJson(
            cluster_polygons[label],
            style_function=lambda x: {
                "fillColor": "red",
                "color": None,
                "fillOpacity": 0.6,
            }
        ).add_to(folium_map)

    return folium_map


In [15]:
import pickle

# Load a pickle file
with open('/content/drive/My Drive/fri/E7/unreachable_clusters_400_H.pkl', 'rb') as f:
    unreachable_clusters_400_H = pickle.load(f)

# Now unreachable_clusters_700_H is available as a Python object


In [19]:
# Call the function to plot top 5 clusters
m_top5 = plot_top_clusters_fixed(
    city_center=(46.236, 14.356),  # Kranj center
    city_graph=city_graph,
    house_nodes=house_nodes,
    unreachable_clusters=unreachable_clusters_400_H,
    new_cutoff=500,    # not used in this version, but could be for extensions
    min_houses=1,      # minimum houses to consider a cluster
    top_k=5,           # show top 5 clusters
    colors_list=['yellow', 'purple', 'white', 'orange', 'green']
)

# Display the map in notebook
m_top5


In [33]:
# Call the function to plot top 5 clusters
m_top5 = plot_top_clusters_fixed(
    city_center=(46.236, 14.356),  # Kranj center
    city_graph=city_graph,
    house_nodes=Cluster_300_obj.house_nodes,
    unreachable_clusters=unreachable_clusters_400_H,
    new_cutoff=500,    # not used in this version, but could be for extensions
    min_houses=1,      # minimum houses to consider a cluster
    top_k=5,           # show top 5 clusters
    colors_list=['yellow', 'purple', 'white', 'orange', 'green']
)

# Display the map in notebook
m_top5


In [28]:
import folium
from folium.plugins import MarkerCluster

def overlay_top_cluster_centers_clustered(
    city_center,
    city_graph,
    house_nodes,
    cluster_dicts,
    cutoff_labels=None,
    min_houses=1,
    top_k=5,
    colors=None
):
    if colors is None:
        colors = ['black', 'blue', 'green', 'purple']
    if cutoff_labels is None:
        cutoff_labels = [str(i) for i in range(len(cluster_dicts))]

    fol_map = folium.Map(location=city_center, zoom_start=12)
    marker_cluster = MarkerCluster().add_to(fol_map)

    for idx, clusters in enumerate(cluster_dicts):
        cluster_house_counts = {}
        for label, cluster in clusters.items():
            nodes_in_cluster = [n for n in cluster['nodes'] if n in city_graph.nodes]
            houses_covered = [h for h in house_nodes if h in nodes_in_cluster]
            if len(houses_covered) >= min_houses:
                cluster_house_counts[label] = len(houses_covered)

        top_clusters = sorted(cluster_house_counts.items(), key=lambda x: x[1], reverse=True)[:top_k]

        print(f"\nCutoff {cutoff_labels[idx]} top clusters:")
        for i, (label, count) in enumerate(top_clusters, start=1):
            print(f"  #{i} color {colors[idx % len(colors)]}: Cluster {label} → {count} new houses")
            center_node = clusters[label]['center_node']
            if center_node not in city_graph.nodes:
                continue

            folium.Marker(
                location=[city_graph.nodes[center_node]['y'], city_graph.nodes[center_node]['x']],
                icon=folium.Icon(color=colors[idx % len(colors)], icon='info-sign'),
                popup=f"Cutoff: {cutoff_labels[idx]}<br>Cluster #{i}<br>New houses: {count}"
            ).add_to(marker_cluster)

    return fol_map

# Usage
m_centers = overlay_top_cluster_centers_clustered(
    city_center=(46.236, 14.356),
    city_graph=city_graph,
    house_nodes=house_nodes,
    cluster_dicts=[
        unreachable_clusters_300_H,
        unreachable_clusters_400_H,
        unreachable_clusters_500_H,
        unreachable_clusters_700_H
    ],
    cutoff_labels=['300m', '400m', '500m', '700m'],
    min_houses=1,
    top_k=5
)

m_centers



Cutoff 300m top clusters:
  #1 color black: Cluster 93 → 19 new houses
  #2 color black: Cluster 69 → 18 new houses
  #3 color black: Cluster 37 → 17 new houses
  #4 color black: Cluster 25 → 13 new houses
  #5 color black: Cluster 84 → 10 new houses

Cutoff 400m top clusters:
  #1 color blue: Cluster 56 → 24 new houses
  #2 color blue: Cluster 63 → 24 new houses
  #3 color blue: Cluster 22 → 18 new houses
  #4 color blue: Cluster 45 → 9 new houses
  #5 color blue: Cluster 5 → 8 new houses

Cutoff 500m top clusters:
  #1 color green: Cluster 37 → 25 new houses
  #2 color green: Cluster 20 → 21 new houses
  #3 color green: Cluster 31 → 18 new houses
  #4 color green: Cluster 27 → 9 new houses
  #5 color green: Cluster 4 → 5 new houses

Cutoff 700m top clusters:
  #1 color purple: Cluster 10 → 22 new houses
  #2 color purple: Cluster 14 → 9 new houses
  #3 color purple: Cluster 1 → 7 new houses
  #4 color purple: Cluster 17 → 7 new houses
  #5 color purple: Cluster 19 → 6 new houses
